In [4]:
import scipy.cluster.vq as vq
import numpy as np
import numpy.linalg as la
import numpy.random as npr
import random as pr
npa = np.array

import sys; sys.path.append('.')
import pdb

#import matplotlib
import pylab
from normal import Normal


def mean(self):
        return np.sum([self.priors[i] * self.comps[i].mean() for i in range(self.ncomps)], axis=0)

def covariance(self): # computed using Dan's method
        m = self.mean()
        s = -np.outer(m,m)

        for i in range(self.ncomps):
            cm = self.comps[i].mean()
            cvar = self.comps[i].covariance()
            s += self.priors[i] * (np.outer(cm,cm) + cvar)

        return s

def pdf(self, x):
        responses = [comp.pdf(x) for comp in self.comps]
        return np.dot(self.priors, responses)

def condition(self, indices, x):
        """
        Create a new GMM conditioned on data x at indices.
        """
        condition_comps = []
        marginal_comps = []

        for comp in self.comps:
            condition_comps.append(comp.condition(indices, x))
            marginal_comps.append(comp.marginalize(indices))

        new_priors = []
        for (i,prior) in enumerate(self.priors):
            new_priors.append(prior * marginal_comps[i].pdf(x))
        new_priors = npa(new_priors) / np.sum(new_priors)

        params = {'ncomps' : self.ncomps, 'comps' : condition_comps,
                  'priors' : new_priors, 'dim' : marginal_comps[0].dim}

        return GMM(params = params)

def em(self, data, nsteps = 100):

        k = self.ncomps
        d = self.dim
        n = len(data)

        for l in range(nsteps):

            # E step

            responses = np.zeros((k,n))

            for j in range(n):
                for i in range(k):
                    responses[i,j] = self.priors[i] * self.comps[i].pdf(data[j])

            responses = responses / np.sum(responses,axis=0) # normalize the weights

            # M step

            N = np.sum(responses,axis=1)

            for i in range(k):
                mu = np.dot(responses[i,:],data) / N[i]
                sigma = np.zeros((d,d))

                for j in range(n):
                   sigma += responses[i,j] * np.outer(data[j,:] - mu, data[j,:] - mu)

                sigma = sigma / N[i]

                self.comps[i].update(mu,sigma) # update the normal with new parameters
                self.priors[i] = N[i] / np.sum(N) # normalize the new priors


def shownormal(data,gmm):

    xnorm = data[:,0]
    ynorm = data[:,1]

    # Plot the normalized faithful data points.
    fig = pylab.figure(num = 1, figsize=(4,4))
    axes = fig.add_subplot(111)
    axes.plot(xnorm,ynorm, '+')

    # Plot the ellipses representing the principle components of the normals.
    for comp in gmm.comps:
        comp.patch(axes)

    pylab.draw()
    pylab.show()


if __name__ == '__main__':

    """
    Tests for gmm module.
    """


    # x = npr.randn(20, 2)

    # print "No data"
    # gmm = GMM(2,1,2) # possibly also broken
    # print gmm

    # print "Uniform"
    # gmm = GMM(2,1,2,data = x, method = "uniform")
    # print gmm

    # print "Random"
    # gmm = GMM(2,1,2,data = x, method = "random") # broken
    # print gmm

    # print "Kmeans"
    # gmm = GMM(2,1,2,data = x, method = "kmeans") # possibly broken
    # print gmm


    x = np.arange(-10,30)
    #y = x ** 2 + npr.randn(20)
    y = x + npr.randn(40) # simple linear function
    #y = np.sin(x) + npr.randn(20)
    data = np.vstack([x,y]).T
    print data.shape


    gmm = GMM(dim = 2, ncomps = 4,data = data, method = "random")
    print gmm
    shownormal(data,gmm)

    gmm.em(data,nsteps=1000)
    shownormal(data,gmm)
    print gmm
    ngmm = gmm.condition([0],[-3])
    print ngmm.mean()
    print ngmm.covariance()

ImportError: No module named normal

In [6]:
x = np.arange(-10,30)
    #y = x ** 2 + npr.randn(20)
y = x + npr.randn(40) # simple linear function
    #y = np.sin(x) + npr.randn(20)
data = np.vstack([x,y]).T
print data

[[-10.          -9.27835605]
 [ -9.          -9.70765337]
 [ -8.          -7.78576817]
 [ -7.          -7.65794737]
 [ -6.          -5.38270777]
 [ -5.          -5.99817738]
 [ -4.          -3.61278249]
 [ -3.          -5.14083632]
 [ -2.          -1.36809239]
 [ -1.          -0.33032718]
 [  0.           0.48412546]
 [  1.           2.5782447 ]
 [  2.           0.74357649]
 [  3.           3.11266893]
 [  4.           3.16438675]
 [  5.           4.61113257]
 [  6.           6.15707957]
 [  7.           6.29493824]
 [  8.           8.7144381 ]
 [  9.           8.24434588]
 [ 10.          10.04254735]
 [ 11.          11.53865545]
 [ 12.          12.64238919]
 [ 13.          11.96753379]
 [ 14.          14.36565203]
 [ 15.          14.00596766]
 [ 16.          16.09949063]
 [ 17.          18.69052775]
 [ 18.          18.17338754]
 [ 19.          20.56445051]
 [ 20.          21.20426159]
 [ 21.          19.13222687]
 [ 22.          23.72677947]
 [ 23.          23.26749099]
 [ 24.        